In [3]:
import pandas as pd
import numpy as np

import pyspark
import re

In [17]:
aj = pd.read_excel('dataset.xlsx')


In [23]:
aj.replace(to_replace=r'^[A-Z]',value='',inplace=True)
aj[aj.columns[0]] = aj[aj.columns[0]].astype(int)

ValueError: invalid literal for int() with base 10: 'C536379'

In [59]:
test = pd.read_csv('clean.csv')
test.info()
dropped = test.select_dtypes(include='int')
st = ''
for i in dropped.columns:
    st+=f' {i} < 0 or'
st = st[:-2]

drop_index = dropped.query(st).index

drop_df = test.drop(index=drop_index)

drop_df.query(f'quantity < 0 or customerid < 0')

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country


In [1]:
from pyspark.sql import SparkSession 
  
spark = SparkSession.builder.appName("DataFrame").getOrCreate() 

In [3]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load('data/clean.csv')

In [4]:
display(df.show())

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|invoiceno|stockcode|         description|quantity|        invoicedate|unitprice|customerid|       country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS S

None

In [5]:
df.columns

['invoiceno',
 'stockcode',
 'description',
 'quantity',
 'invoicedate',
 'unitprice',
 'customerid',
 'country']

# Ganti nama kolom

In [6]:
df = df.toDF(*[i.upper() for i in df.columns])

In [13]:
display(df.head(10))

[Row(invoiceno='536365', stockcode='85123A', description='WHITE HANGING HEART T-LIGHT HOLDER', quantity=6, invoicedate=datetime.datetime(2010, 12, 1, 8, 26), unitprice=2.55, customerid=17850, country='United Kingdom'),
 Row(invoiceno='536365', stockcode='71053', description='WHITE METAL LANTERN', quantity=6, invoicedate=datetime.datetime(2010, 12, 1, 8, 26), unitprice=3.39, customerid=17850, country='United Kingdom'),
 Row(invoiceno='536365', stockcode='84406B', description='CREAM CUPID HEARTS COAT HANGER', quantity=8, invoicedate=datetime.datetime(2010, 12, 1, 8, 26), unitprice=2.75, customerid=17850, country='United Kingdom'),
 Row(invoiceno='536365', stockcode='84029G', description='KNITTED UNION FLAG HOT WATER BOTTLE', quantity=6, invoicedate=datetime.datetime(2010, 12, 1, 8, 26), unitprice=3.39, customerid=17850, country='United Kingdom'),
 Row(invoiceno='536365', stockcode='84029E', description='RED WOOLLY HOTTIE WHITE HEART.', quantity=6, invoicedate=datetime.datetime(2010, 12, 

# Melihat Tipe Data

In [7]:
def info(tsar):
    for i in range(len(tsar.columns)):
        print(f"{tsar.columns[i]} =",tsar.schema[tsar.columns[i]].dataType)

# Mengubah Tipe Data

In [8]:
from pyspark.sql.types import DoubleType, IntegerType, StringType

# for i in range(len(df.columns)):
#     df = df.withColumn(df.columns[i],df[df.columns[i]].cast(StringType()))

info(df)

invoiceno = StringType()
stockcode = StringType()
description = StringType()
quantity = IntegerType()
invoicedate = TimestampType()
unitprice = DoubleType()
customerid = IntegerType()
country = StringType()


In [10]:
test = df

In [11]:
info(test)

invoiceno = StringType()
stockcode = StringType()
description = StringType()
quantity = IntegerType()
invoicedate = TimestampType()
unitprice = DoubleType()
customerid = IntegerType()
country = StringType()


# Membaca File Excel

In [12]:
import pyspark.pandas as ps

c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [13]:
dataframe = df.toPandas()

In [14]:
dataframe.columns = [i.lower() for i in dataframe.columns]
dataframe

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
536636,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
536637,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.1,12680,France
536638,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
536639,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [19]:
import pkg_resources
pkg_resources.get_distribution('crealytics')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_20820\825632357.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


DistributionNotFound: The 'crealytics' distribution was not found and is required by the application

In [18]:
spark1 = SparkSession.builder.appName("PANDAS").config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.7").getOrCreate()

pdf = spark1.read.format("com.crealytics.spark.excel").options(header='True',inferSchema='True').load('dataset.xlsx')

Py4JJavaError: An error occurred while calling o147.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: com.crealytics.spark.excel. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:186)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:75)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMetho